In [15]:
using ITensors,ITensorMPS
using RandomMeas
using ProgressBars

In [16]:
N = 4
ξ = siteinds("Qubit", N)
B = 1.
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising ZZ interactions
  ampo .+= -1, "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field X
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ)
H2 = apply(H,H)
# Density-matrix renormalization group
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = randomMPS(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)
# Run 
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψ = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-4.750830495034999  maxlinkdim=4 maxerr=1.27E-16 time=0.013
After sweep 2 energy=-4.758758080920338  maxlinkdim=4 maxerr=2.85E-16 time=0.007
After sweep 3 energy=-4.758770462706213  maxlinkdim=4 maxerr=1.22E-16 time=0.007
After sweep 4 energy=-4.7587704831097675  maxlinkdim=4 maxerr=2.23E-16 time=0.008
After sweep 5 energy=-4.758770483143578  maxlinkdim=4 maxerr=0.00E+00 time=0.007

Ground state energy:  -4.758770483143578

---------------------------------------



In [17]:
#Consider experimental state admixed with white noise
p = 0.05
Ide = MPO(ξ)
for i in 1:N
    Ide[i] = δ(ξ[i]',ξ[i])
end
ρ = (1-p)*outer(ψ',ψ)+p*Ide/2^N;

In [18]:
nu = 500
NM = 100
data = zeros(Int8,(nu,NM,N));

In [19]:
Es = 0.
E2s = 0.
n = 3 #We will create three batch shadows
u = Vector{Vector{ITensor}}()
ITensors.disable_warn_order()
for r in ProgressBar(1:nu, printing_delay=20)
        push!(u,get_rotations(ξ,1))
        data[r,:,:] =  get_RandomMeas(ρ,u[r],NM)
end

0.0%┣                                              ┫ 0/500 [00:00<00:00, -0s/it]
100.0%┣█████████████████████████████████████████┫ 500/500 [00:01<00:00, 338it/s]
100.0%┣█████████████████████████████████████████┫ 500/500 [00:01<00:00, 338it/s]


In [20]:
ρb = get_batch_shadows(data,ξ,u,n)
ρm = Vector{ITensor}(undef,n)
ρm[1] = (ρb[1]+ρb[2]+ρb[3])/3/nu
ρm[2] = (multiply(ρb[1],ρb[2])+multiply(ρb[1],ρb[3])+multiply(ρb[2],ρb[3]))/3/nu
ρm[3] = multiply(multiply(ρb[1],ρb[2]),ρb[3])/nu;

In [21]:
Es = zeros(n)
for nt in 1:n
    Es[nt] = real(trace(multiply(ρm[nt],flatten(H)),ξ)/trace(ρm[nt],ξ))
end

In [22]:
println("Estimations for different purification order n " , Es)
println("Real values ",E)

Estimations for different purification order n [-4.630577855410757, -4.745252245834473, -4.752183665255631]
Real values -4.758770483143578
